TODO
- add code to drop undesirable variables for goalkeepers

In [1]:
# number of latest gameweek
gameweek = 25

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt
import pickle
import json
import requests

import shap
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from scipy.stats import poisson
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# allow more data columns/rows to be shown than by default
pd.set_option('display.max_columns', 501)
pd.set_option('display.max_rows', 501)

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)
#display(player_name_dict)

In [4]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
display(fpl_df.head(5))

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,NaN,NaN,84450,0,0,-1,1,1,3,3.3,5.6,6,Granit,2.3,3,False,NaN,NaN,49,84450.jpg,3.7,Xhaka,2.7,False,NaN,a,Arsenal,3,93,843993,2257,731230,10159,0.5,19.0,Xhaka,2107,3,5,10,23,0,0,0,3,0,0,7,406,448.6,524.3,340.0,131.4,25,2.20,3.07,5.27,23.13,54,21,17,14,65,42,31,21,NaN,NaN,3.0,NaN,NaN,NaN,0.09,0.0,0.13,0.23,0.99,0.98,232,132,153,66,78,32,113,38,1.07,0.43,2022-23,Granit Xhaka,MID,25,4.9,303765.39,-7902,NaN,NaN,1.0
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,0.0,0,Mohamed,0.0,4,False,Knee injury - Unknown return date,2023-01-26T10:00:07.778526Z,41,153256.jpg,1.2,Elneny,0.6,False,NaN,i,Arsenal,3,6,198169,276,269899,1551,0.0,1.5,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,479,219,431,212,555,224,482,223,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.03,0.03,1.05,1.62,624,321,497,178,408,179,233,72,0.81,0.00,2022-23,Mohamed Elneny,MID,25,4.1,67503.42,-1275,NaN,NaN,1.0
2,NaN,NaN,156074,0,0,-3,3,0,2,1.2,1.4,0,Rob,0.2,5,False,NaN,NaN,42,156074.jpg,1.0,Holding,0.1,False,NaN,a,Arsenal,3,7,7112,58,18308,112,0.0,1.7,Holding,17,0,0,0,0,0,0,0,0,0,0,0,23,3.4,0.1,0.0,0.3,0,0.00,0.00,0.00,0.28,485,172,490,177,561,222,503,177,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.00,1.48,0.00,595,169,348,123,440,162,435,162,0.00,0.00,2022-23,Rob Holding,DEF,25,4.2,11250.57,-54,NaN,NaN,1.0
3,100.0,75.0,167199,0,0,-3,3,0,3,2.0,2.2,2,Thomas,1.0,6,False,NaN,2023-02-15T22:30:09.735511Z,47,167199.jpg,3.0,Partey,0.4,False,NaN,a,Arsenal,3,59,99934,432,131732,780,0.2,12.6,Partey,1591,2,0,9,11,0,0,0,2,0,0,5,307,353.4,310.8,174.0,84.0,18,1.63,1.36,3.00,18.18,95,40,62,50,145,89,92,64,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.0,0.08,0.17,1.03,0.62,315,191,266,121,157,68,278,91,1.02,0.51,2022-23,Thomas Partey,MID,25,4.7,45002.28,-348,NaN,NaN,1.0
4,100.0,100.0,184029,0,0,5,-5,4,3,5.2,9.4,11,Martin,4.2,7,True,NaN,2022-09-18T20:00:06.315087Z,70,184029.jpg,5.9,Ødegaard,38.3,False,NaN,a,Arsenal,3,142,5377123,78836,2823078,108051,0.6,20.3,Ødegaard,2041,9,7,10,22,0,0,0,3,0,0,21,542,619.6,810.4,632.0,206.4,24,6.57,5.56,12.17,23.29,14,5,5,4,19,11,8,6,3.0,NaN,1.0,NaN,NaN,NaN,0.29,0.0,0.25,0.54,1.03,0.97,40,20,43,15,8,3,5,3,1.06,0.44,2022-23,Martin degaard,MID,25,7.0,4308968.31,-29215,NaN,NaN,1.0


In [5]:
fpl_fixtures = pd.read_csv('../data/fpl_fixtures.csv', index_col=0)
display(fpl_fixtures.head(5))

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2293083,NaN,False,False,274,NaN,0,False,NaN,14,NaN,5,NaN,[],3,3,75184,Brighton,Manchester Utd
1,2293085,NaN,False,False,276,NaN,0,False,NaN,9,NaN,12,NaN,[],2,4,75186,Liverpool,Fulham
2,2293086,NaN,False,False,277,NaN,0,False,NaN,19,NaN,13,NaN,[],2,5,75187,Manchester City,West Ham
3,2293057,NaN,False,False,248,NaN,0,False,NaN,5,NaN,15,NaN,[],3,4,75158,Newcastle Utd,Brighton
4,2292810,1.0,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911,Crystal Palace,Arsenal


In [6]:
team_stats = pd.read_csv('../data/team_stats.csv', index_col=0, usecols=['date','team','team_xG', 'team_xGA'])
team_stats.head(5)

,team,team_xG,team_xGA
date,,,
2017-08-11 00:00:00+00:00,Arsenal,2.3,1.3
2017-08-11 00:00:00+00:00,Leicester City,1.3,2.3
2017-08-12 00:00:00+00:00,Watford,2.1,3.1
2017-08-12 00:00:00+00:00,Liverpool,3.1,2.1
2017-08-12 00:00:00+00:00,Crystal Palace,1.1,1.7


In [7]:
xgb_models = []
for i in range(1,11):
    xgb = XGBRegressor()
    xgb.load_model(f'../models/xgb_regression_full_game{i}.json')
    xgb_models.append(xgb)
display(xgb_models[0])
display(xgb_models[9])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8254703688775018,
             enable_categorical=False, gamma=0.25226144515542404, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.06285391203893921, max_delta_step=0, max_depth=3,
             min_child_weight=1.8725484088145359, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7993913128492731, tree_method='auto',
             validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5195376619139985,
             enable_categorical=False, gamma=0.35139253750058574, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.03261313301598562, max_delta_step=0, max_depth=3,
             min_child_weight=0.9469696895323456, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.804844605129602, tree_method='auto',
             validate_parameters=1, verbosity=None)

In [8]:
player_data = pd.read_csv('../data/data_for_projections.csv', index_col=0)
player_data['name'] = player_data['name'].map(player_name_dict).fillna(player_data['name'])
player_data.head(5)

,GW,assists,bps,penalties_saved,goals_conceded,selected,goals_scored,yellow_cards,own_goals,value,clean_sheets,position,penalties_missed,red_cards,creativity,minutes,ict_index,saves,total_points,opp_team_name,transfers_in,season,kickoff_time,was_home,name,bonus,influence,transfers_out,threat,transfers_balance,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,General_xAG,Pass Types_TI,Dribbles_Mis,Dribbles_Dis,Passes_PrgP,Carries_PrgC,Take-Ons_Att,Take-Ons_Succ,General_PrgP,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Receiving_PrgR,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD,fbref_position_GK,fbref_position_LB,fbref_position_CB,fbref_position_RB,fbref_position_WB,fbref_position_DM,fbref_position_LM,fbref_position_CM,fbref_position_RM,fbref_position_AM,fbref_position_LW,fbref_position_RW,fbref_position_FW,team_xG,team_xGA,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG,opp_team_xGA,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0,xPoints,games_in_past_5_days_shifted_0,games_in_past_10_days_shifted_0,games_in_past_20_days_shifted_0,games_in_past_40_days_shifted_0,selected_shifted_0,transfers_balance_shifted_0,value_shifted_0,transfers_balance_%_shifted_0,transfers_balance_%_ewa5_shifted_0,transfers_balance_%_ewa10_shifted_0,value_change_1_shifted_0,value_change_2_shifted_0,value_change_5_shifted_0,goal_to_xg_ratio_ewa_40_shifted0,total_points_ewa_5_shifted_0,xPoints_ewa_5_shifted_0,assists_ewa_5_shifted_0,bonus_ewa_5_shifted_0,bps_ewa_5_shifted_0,clean_sheets_ewa_5_shifted_0,creativity_ewa_5_shifted_0,goals_conceded_ewa_5_shifted_0,goals_scored_ewa_5_shifted_0,ict_index_ewa_5_shifted_0,influence_ewa_5_shifted_0,minutes_ewa_5_shifted_0,own_goals_ewa_5_shifted_0,penalties_missed_ewa_5_shifted_0,pen

In [9]:
keep = [
        'was_home', 'fpl_position_GK', 'fpl_position_DEF',
       'fpl_position_MID', 'fpl_position_FWD', 'team_xG_ewa_5_shifted_0',
       'team_xG_ewa_10_shifted_0', 'team_xG_ewa_20_shifted_0',
       'team_xG_ewa_40_shifted_0', 'team_xGA_ewa_5_shifted_0',
       'team_xGA_ewa_10_shifted_0', 'team_xGA_ewa_20_shifted_0',
       'team_xGA_ewa_40_shifted_0', 'opp_team_xG_ewa_5_shifted_0',
       'opp_team_xG_ewa_10_shifted_0', 'opp_team_xG_ewa_20_shifted_0',
       'opp_team_xG_ewa_40_shifted_0', 'opp_team_xGA_ewa_5_shifted_0',
       'opp_team_xGA_ewa_10_shifted_0', 'opp_team_xGA_ewa_20_shifted_0',
       'opp_team_xGA_ewa_40_shifted_0', 'games_in_past_5_days_shifted_0',
       'games_in_past_10_days_shifted_0',
       'games_in_past_20_days_shifted_0',
       'games_in_past_40_days_shifted_0',
       'transfers_balance_%_shifted_0',
       'transfers_balance_%_ewa5_shifted_0',
       'transfers_balance_%_ewa10_shifted_0',
       'value_change_1_shifted_0', 'value_change_2_shifted_0',
       'value_change_5_shifted_0', 'goal_to_xg_ratio_ewa_40_shifted0',
       'total_points_ewa_5_shifted_0', 'xPoints_ewa_5_shifted_0',
       'assists_ewa_5_shifted_0', 'bonus_ewa_5_shifted_0',
       'bps_ewa_5_shifted_0', 'clean_sheets_ewa_5_shifted_0',
       'creativity_ewa_5_shifted_0', 'goals_conceded_ewa_5_shifted_0',
       'goals_scored_ewa_5_shifted_0', 'ict_index_ewa_5_shifted_0',
       'influence_ewa_5_shifted_0', 'minutes_ewa_5_shifted_0',
       'own_goals_ewa_5_shifted_0', 'penalties_missed_ewa_5_shifted_0',
       'penalties_saved_ewa_5_shifted_0', 'saves_ewa_5_shifted_0',
       'threat_ewa_5_shifted_0', 'yellow_cards_ewa_5_shifted_0',
       'Performance_Sh_ewa_5_shifted_0',
       'Performance_SoT_ewa_5_shifted_0',
       'Performance_Touches_ewa_5_shifted_0',
       'Performance_Press_ewa_5_shifted_0',
       'Performance_Tkl_ewa_5_shifted_0',
       'Performance_Int_ewa_5_shifted_0',
       'Performance_Blocks_ewa_5_shifted_0',
       'Expected_xG_ewa_5_shifted_0', 'Expected_npxG_ewa_5_shifted_0',
       'Expected_xA_ewa_5_shifted_0', 'SCA_SCA_ewa_5_shifted_0',
       'Passes_Cmp_ewa_5_shifted_0', 'Passes_Cmp%_ewa_5_shifted_0',
       'Passes_Prog_ewa_5_shifted_0', 'Carries_Carries_ewa_5_shifted_0',
       'Carries_Prog_ewa_5_shifted_0', 'Dribbles_Succ_ewa_5_shifted_0',
       'Dribbles_Att_ewa_5_shifted_0', 'General_KP_ewa_5_shifted_0',
       'Touches_Att 3rd_ewa_5_shifted_0',
       'Touches_Att Pen_ewa_5_shifted_0',
       'Performance_Recov_ewa_5_shifted_0',
       'Aerial Duels_Won_ewa_5_shifted_0',
       'Aerial Duels_Won%_ewa_5_shifted_0',
       'fbref_position_GK_ewa5_shifted_0',
       'fbref_position_LB_ewa5_shifted_0',
       'fbref_position_CB_ewa5_shifted_0',
       'fbref_position_RB_ewa5_shifted_0',
       'fbref_position_WB_ewa5_shifted_0',
       'fbref_position_DM_ewa5_shifted_0',
       'fbref_position_LM_ewa5_shifted_0',
       'fbref_position_CM_ewa5_shifted_0',
       'fbref_position_RM_ewa5_shifted_0',
       'fbref_position_AM_ewa5_shifted_0',
       'fbref_position_LW_ewa5_shifted_0',
       'fbref_position_RW_ewa5_shifted_0',
       'fbref_position_FW_ewa5_shifted_0',
       'total_points_ewa_10_shifted_0', 'xPoints_ewa_10_shifted_0',
       'assists_ewa_10_shifted_0', 'bonus_ewa_10_shifted_0',
       'bps_ewa_10_shifted_0', 'clean_sheets_ewa_10_shifted_0',
       'creativity_ewa_10_shifted_0', 'goals_conceded_ewa_10_shifted_0',
       'goals_scored_ewa_10_shifted_0', 'ict_index_ewa_10_shifted_0',
       'influence_ewa_10_shifted_0', 'minutes_ewa_10_shifted_0',
       'own_goals_ewa_10_shifted_0',
       'penalties_missed_ewa_10_shifted_0',
       'penalties_saved_ewa_10_shifted_0', 'saves_ewa_10_shifted_0',
       'threat_ewa_10_shifted_0', 'yellow_cards_ewa_10_shifted_0',
       'Performance_Sh_ewa_10_shifted_0',
       'Performance_SoT_ewa_10_shifted_0',
       'Performance_Touches_ewa_10_shifted_0',
       'Performance_Press_ewa_10_shifted_0',
       'Performance_Tkl_ewa_10_shifted_0',
       'Performance_Int_ewa_10_shifted_0',
       'Performance_Blocks_ewa_10_shifted_0',
       'Expected_xG_ewa_10_shifted_0', 'Expected_npxG_ewa_10_shifted_0',
       'Expected_xA_ewa_10_shifted_0', 'SCA_SCA_ewa_10_shifted_0',
       'Passes_Cmp_ewa_10_shifted_0', 'Passes_Cmp%_ewa_10_shifted_0',
       'Passes_Prog_ewa_10_shifted_0',
       'Carries_Carries_ewa_10_shifted_0',
       'Carries_Prog_ewa_10_shifted_0',
       'Dribbles_Succ_ewa_10_shifted_0',
       'Dribbles_Att_ewa_10_shifted_0', 'General_KP_ewa_10_shifted_0',
       'Touches_Att 3rd_ewa_10_shifted_0',
       'Touches_Att Pen_ewa_10_shifted_0',
       'Performance_Recov_ewa_10_shifted_0',
       'Aerial Duels_Won_ewa_10_shifted_0',
       'Aerial Duels_Won%_ewa_10_shifted_0',
       'fbref_position_GK_ewa10_shifted_0',
       'fbref_position_LB_ewa10_shifted_0',
       'fbref_position_CB_ewa10_shifted_0',
       'fbref_position_RB_ewa10_shifted_0',
       'fbref_position_WB_ewa10_shifted_0',
       'fbref_position_DM_ewa10_shifted_0',
       'fbref_position_LM_ewa10_shifted_0',
       'fbref_position_CM_ewa10_shifted_0',
       'fbref_position_RM_ewa10_shifted_0',
       'fbref_position_AM_ewa10_shifted_0',
       'fbref_position_LW_ewa10_shifted_0',
       'fbref_position_RW_ewa10_shifted_0',
       'fbref_position_FW_ewa10_shifted_0',
       'total_points_ewa_20_shifted_0', 'xPoints_ewa_20_shifted_0',
       'assists_ewa_20_shifted_0', 'bonus_ewa_20_shifted_0',
       'bps_ewa_20_shifted_0', 'clean_sheets_ewa_20_shifted_0',
       'creativity_ewa_20_shifted_0', 'goals_conceded_ewa_20_shifted_0',
       'goals_scored_ewa_20_shifted_0', 'ict_index_ewa_20_shifted_0',
       'influence_ewa_20_shifted_0', 'minutes_ewa_20_shifted_0',
       'own_goals_ewa_20_shifted_0',
       'penalties_missed_ewa_20_shifted_0',
       'penalties_saved_ewa_20_shifted_0', 'saves_ewa_20_shifted_0',
       'threat_ewa_20_shifted_0', 'yellow_cards_ewa_20_shifted_0',
       'Performance_Sh_ewa_20_shifted_0',
       'Performance_SoT_ewa_20_shifted_0',
       'Performance_Touches_ewa_20_shifted_0',
       'Performance_Press_ewa_20_shifted_0',
       'Performance_Tkl_ewa_20_shifted_0',
       'Performance_Int_ewa_20_shifted_0',
       'Performance_Blocks_ewa_20_shifted_0',
       'Expected_xG_ewa_20_shifted_0', 'Expected_npxG_ewa_20_shifted_0',
       'Expected_xA_ewa_20_shifted_0', 'SCA_SCA_ewa_20_shifted_0',
       'Passes_Cmp_ewa_20_shifted_0', 'Passes_Cmp%_ewa_20_shifted_0',
       'Passes_Prog_ewa_20_shifted_0',
       'Carries_Carries_ewa_20_shifted_0',
       'Carries_Prog_ewa_20_shifted_0',
       'Dribbles_Succ_ewa_20_shifted_0',
       'Dribbles_Att_ewa_20_shifted_0', 'General_KP_ewa_20_shifted_0',
       'Touches_Att 3rd_ewa_20_shifted_0',
       'Touches_Att Pen_ewa_20_shifted_0',
       'Performance_Recov_ewa_20_shifted_0',
       'Aerial Duels_Won_ewa_20_shifted_0',
       'Aerial Duels_Won%_ewa_20_shifted_0',
       'fbref_position_GK_ewa20_shifted_0',
       'fbref_position_LB_ewa20_shifted_0',
       'fbref_position_CB_ewa20_shifted_0',
       'fbref_position_RB_ewa20_shifted_0',
       'fbref_position_WB_ewa20_shifted_0',
       'fbref_position_DM_ewa20_shifted_0',
       'fbref_position_LM_ewa20_shifted_0',
       'fbref_position_CM_ewa20_shifted_0',
       'fbref_position_RM_ewa20_shifted_0',
       'fbref_position_AM_ewa20_shifted_0',
       'fbref_position_LW_ewa20_shifted_0',
       'fbref_position_RW_ewa20_shifted_0',
       'fbref_position_FW_ewa20_shifted_0',
       'total_points_ewa_40_shifted_0', 'xPoints_ewa_40_shifted_0',
       'assists_ewa_40_shifted_0', 'bonus_ewa_40_shifted_0',
       'bps_ewa_40_shifted_0', 'clean_sheets_ewa_40_shifted_0',
       'creativity_ewa_40_shifted_0', 'goals_conceded_ewa_40_shifted_0',
       'goals_scored_ewa_40_shifted_0', 'ict_index_ewa_40_shifted_0',
       'influence_ewa_40_shifted_0', 'minutes_ewa_40_shifted_0',
       'own_goals_ewa_40_shifted_0',
       'penalties_missed_ewa_40_shifted_0',
       'penalties_saved_ewa_40_shifted_0', 'saves_ewa_40_shifted_0',
       'threat_ewa_40_shifted_0', 'yellow_cards_ewa_40_shifted_0',
       'Performance_Sh_ewa_40_shifted_0',
       'Performance_SoT_ewa_40_shifted_0',
       'Performance_Touches_ewa_40_shifted_0',
       'Performance_Press_ewa_40_shifted_0',
       'Performance_Tkl_ewa_40_shifted_0',
       'Performance_Int_ewa_40_shifted_0',
       'Performance_Blocks_ewa_40_shifted_0',
       'Expected_xG_ewa_40_shifted_0', 'Expected_npxG_ewa_40_shifted_0',
       'Expected_xA_ewa_40_shifted_0', 'SCA_SCA_ewa_40_shifted_0',
       'Passes_Cmp_ewa_40_shifted_0', 'Passes_Cmp%_ewa_40_shifted_0',
       'Passes_Prog_ewa_40_shifted_0',
       'Carries_Carries_ewa_40_shifted_0',
       'Carries_Prog_ewa_40_shifted_0',
       'Dribbles_Succ_ewa_40_shifted_0',
       'Dribbles_Att_ewa_40_shifted_0', 'General_KP_ewa_40_shifted_0',
       'Touches_Att 3rd_ewa_40_shifted_0',
       'Touches_Att Pen_ewa_40_shifted_0',
       'Performance_Recov_ewa_40_shifted_0',
       'Aerial Duels_Won_ewa_40_shifted_0',
       'Aerial Duels_Won%_ewa_40_shifted_0',
       'fbref_position_GK_ewa40_shifted_0',
       'fbref_position_LB_ewa40_shifted_0',
       'fbref_position_CB_ewa40_shifted_0',
       'fbref_position_RB_ewa40_shifted_0',
       'fbref_position_WB_ewa40_shifted_0',
       'fbref_position_DM_ewa40_shifted_0',
       'fbref_position_LM_ewa40_shifted_0',
       'fbref_position_CM_ewa40_shifted_0',
       'fbref_position_RM_ewa40_shifted_0',
       'fbref_position_AM_ewa40_shifted_0',
       'fbref_position_LW_ewa40_shifted_0',
       'fbref_position_RW_ewa40_shifted_0',
       'fbref_position_FW_ewa40_shifted_0'
]
print(len(keep))

252


In [10]:
team_features = [
        'team_xG_ewa_5_shifted_0',
       'team_xG_ewa_10_shifted_0', 'team_xG_ewa_20_shifted_0',
       'team_xG_ewa_40_shifted_0', 'team_xGA_ewa_5_shifted_0',
       'team_xGA_ewa_10_shifted_0', 'team_xGA_ewa_20_shifted_0',
       'team_xGA_ewa_40_shifted_0'
]
opponent_features = [
    'opp_team_xG_ewa_5_shifted_0',
       'opp_team_xG_ewa_10_shifted_0', 'opp_team_xG_ewa_20_shifted_0',
       'opp_team_xG_ewa_40_shifted_0', 'opp_team_xGA_ewa_5_shifted_0',
       'opp_team_xGA_ewa_10_shifted_0', 'opp_team_xGA_ewa_20_shifted_0',
       'opp_team_xGA_ewa_40_shifted_0'
]

In [11]:
teams = fpl_fixtures['home_team'].unique()
teams

array(['Brighton', 'Liverpool', 'Manchester City', 'Newcastle Utd',
       'Crystal Palace', 'Fulham', 'Bournemouth', 'Leeds United',
       'Tottenham', 'Everton', 'Leicester City', 'Manchester Utd',
       'West Ham', 'Aston Villa', 'Arsenal', 'Southampton', 'Wolves',
       'Brentford', 'Nottingham Forest', 'Chelsea'], dtype=object)

In [12]:
team_stats_for_predictions = pd.DataFrame(columns=[team_features + opponent_features])
for team in teams:
    team_stats_for_predictions.loc[team, 'team_xG_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/40).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/40).mean().values[-1]
    
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/40).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/40).mean().values[-1]

aux_columns = ['opp_team_xG_ewa_5_shifted_0', 'opp_team_xG_ewa_10_shifted_0','opp_team_xG_ewa_20_shifted_0',
               'opp_team_xG_ewa_40_shifted_0']
team_stats_for_predictions.loc['Nottingham Forest', aux_columns] = team_stats_for_predictions.loc['Fulham', aux_columns].copy()
aux_columns = ['opp_team_xGA_ewa_5_shifted_0', 'opp_team_xGA_ewa_10_shifted_0','opp_team_xGA_ewa_20_shifted_0',
               'opp_team_xGA_ewa_40_shifted_0']
team_stats_for_predictions.loc['Nottingham Forest', aux_columns] = team_stats_for_predictions.loc['Bournemouth', aux_columns].copy()

team_stats_for_predictions.head(5)

,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0
Brighton,1.916217,1.720602,1.562545,1.419852,1.073743,1.174004,1.184846,1.191693,1.916217,1.720602,1.562545,1.419852,1.073743,1.174004,1.184846,1.191693
Liverpool,1.879459,1.860433,1.917054,1.972925,1.205426,1.273526,1.214206,1.13976,1.879459,1.860433,1.917054,1.972925,1.205426,1.273526,1.214206,1.13976
Manchester City,2.245588,2.192445,2.198598,2.212216,0.986665,0.914461,0.845087,0.800701,2.245588,2.192445,2.198598,2.212216,0.986665,0.914461,0.845087,0.800701
Newcastle Utd,1.562754,1.561279,1.460305,1.303929,1.232272,1.089573,1.119319,1.224136,1.562754,1.561279,1.460305,1.303929,1.232272,1.089573,1.119319,1.224136
Crystal Palace,0.868536,0.911598,0.979855,1.016025,1.39286,1.37017,1.312973,1.298913,0.868536,0.911598,0.979855,1.016025,1.39286,1.37017,1.312973,1.298913


In [13]:
player_names = fpl_df.loc[fpl_df['minutes']>0, 'name'].unique()
players_not_projected = []
for player_name in player_names:   
    if player_data[player_data['name']==player_name].shape[0]==0:
        players_not_projected.append(player_name)
display(players_not_projected)
display(len(players_not_projected))

['Fabio Ferreira Vieira',
 'Marcus Oliveira Alencar',
 'Jorge Luiz Frello Filho',
 'Diego Carlos Santos Silva',
 'Alexandre Moreno Lopera',
 'Norberto Murara Neto',
 'Hamed Traore',
 'Sergi Canos Tenes',
 'Mads Roerslev Rasmussen',
 'Mathias Jorgensen',
 'Jeremy Sarmiento Morante',
 'Julio Enciso',
 'Omari Hutchinson',
 'David Datro Fofana',
 'Cedric Alves Soares',
 'Carlos Vinicius Alves Morais',
 'Ricardo Barbosa Pereira',
 'Victor Kristiansen',
 'Mateus Cardoso Lemos Martins',
 'Junior Firpo Adames',
 'Mateo Joseph Fernandez',
 'Fabio Henrique Tavares',
 'Frederico Rodrigues de Paula Santos',
 'Facundo Pellistri Rebollo',
 'Javier Manquillo Gaitan',
 'Carlos Ribeiro Dias',
 'Alex Mighten',
 'Renan Augusto Lodi dos Santos',
 'Gustavo Henrique Furtado Scarpa',
 'Danilo dos Santos de Oliveira',
 'Felipe Augusto de Almeida Monteiro',
 'Lyanco Silveira Neves Vojnovic',
 'Armel Bella-Kotchap',
 'Juan Larios Lopez',
 'Emerson Leite de Souza Junior',
 'Bryan Gil Salvatierra',
 'Emerson Palm

41

In [14]:
%%time
# run xP projections for each player
first_projected_gameweek=gameweek+1
horizon=10
player_names = fpl_df['name'].unique()
all_predictions = []
prediction_data = pd.DataFrame()
players_not_projected = []
for player_name in player_names:   
    if player_data[player_data['name']==player_name].shape[0]>0:
        team = fpl_df.loc[fpl_df['name']==player_name, 'team'].values[0]
        #price = fpl_df.loc[fpl_df['name']==player_name, 'now_cost'].values[0] / 10.0
        aux = player_data[player_data['name']==player_name].tail(1)
        for count, week in enumerate(np.arange(first_projected_gameweek, first_projected_gameweek+horizon)):
            home_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['home_team']==team)]
            away_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['away_team']==team)]
            for ix in home_games.index:
                aux['was_home'] = 1                
                aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
                opp_team = fpl_fixtures.loc[ix, 'away_team']
                aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()                
                prediction_data = pd.concat([prediction_data, aux])
                xP = xgb_models[count].predict(aux[keep])[0]
                all_predictions.append([player_name, team, opp_team, 1, week, xP])
            for ix in away_games.index:
                aux['was_home'] = 0
                aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
                opp_team = fpl_fixtures.loc[ix, 'home_team']
                aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
                prediction_data = pd.concat([prediction_data, aux])
                xP = xgb_models[count].predict(aux[keep])[0]
                all_predictions.append([player_name, team, opp_team, 0, week, xP])
    else:
        players_not_projected.append(player_name)

my_columns = ['name', 'team', 'opp_team', 'was_home', 'gameweek', 'xP']
predictions = pd.DataFrame(all_predictions, columns=my_columns)
prediction_data = prediction_data.reset_index(drop=True)
display(prediction_data.head(5))

,GW,assists,bps,penalties_saved,goals_conceded,selected,goals_scored,yellow_cards,own_goals,value,clean_sheets,position,penalties_missed,red_cards,creativity,minutes,ict_index,saves,total_points,opp_team_name,transfers_in,season,kickoff_time,was_home,name,bonus,influence,transfers_out,threat,transfers_balance,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,General_xAG,Pass Types_TI,Dribbles_Mis,Dribbles_Dis,Passes_PrgP,Carries_PrgC,Take-Ons_Att,Take-Ons_Succ,General_PrgP,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Receiving_PrgR,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD,fbref_position_GK,fbref_position_LB,fbref_position_CB,fbref_position_RB,fbref_position_WB,fbref_position_DM,fbref_position_LM,fbref_position_CM,fbref_position_RM,fbref_position_AM,fbref_position_LW,fbref_position_RW,fbref_position_FW,team_xG,team_xGA,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG,opp_team_xGA,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0,xPoints,games_in_past_5_days_shifted_0,games_in_past_10_days_shifted_0,games_in_past_20_days_shifted_0,games_in_past_40_days_shifted_0,selected_shifted_0,transfers_balance_shifted_0,value_shifted_0,transfers_balance_%_shifted_0,transfers_balance_%_ewa5_shifted_0,transfers_balance_%_ewa10_shifted_0,value_change_1_shifted_0,value_change_2_shifted_0,value_change_5_shifted_0,goal_to_xg_ratio_ewa_40_shifted0,total_points_ewa_5_shifted_0,xPoints_ewa_5_shifted_0,assists_ewa_5_shifted_0,bonus_ewa_5_shifted_0,bps_ewa_5_shifted_0,clean_sheets_ewa_5_shifted_0,creativity_ewa_5_shifted_0,goals_conceded_ewa_5_shifted_0,goals_scored_ewa_5_shifted_0,ict_index_ewa_5_shifted_0,influence_ewa_5_shifted_0,minutes_ewa_5_shifted_0,own_goals_ewa_5_shifted_0,penalties_missed_ewa_5_shifted_0,pen

CPU times: user 6min 3s, sys: 1min 23s, total: 7min 26s
Wall time: 1min 1s


In [15]:
predictions.head(5)

,name,team,opp_team,was_home,gameweek,xP
0,Granit Xhaka,Arsenal,Bournemouth,1,26,3.824997
1,Granit Xhaka,Arsenal,Fulham,0,27,3.240060
2,Granit Xhaka,Arsenal,Crystal Palace,1,28,3.926446
3,Granit Xhaka,Arsenal,Leeds United,1,29,3.177404
4,Granit Xhaka,Arsenal,Liverpool,0,30,2.469698


In [16]:
aux = predictions[predictions['name'].str.contains('Haaland')].copy()
aux['xP_cumulative'] = aux['xP'].cumsum()
display(aux)
ppg = aux['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')

,name,team,opp_team,was_home,gameweek,xP,xP_cumulative
3759,Erling Haaland,Manchester City,Newcastle Utd,1,26,5.569074,5.569074
3760,Erling Haaland,Manchester City,Crystal Palace,0,27,4.330642,9.899715
3761,Erling Haaland,Manchester City,Liverpool,1,29,4.641608,14.541323
3762,Erling Haaland,Manchester City,Southampton,0,30,4.750614,19.291937
3763,Erling Haaland,Manchester City,Leicester City,1,31,6.014260,25.306198
3764,Erling Haaland,Manchester City,Brighton,0,32,4.092460,29.398659
3765,Erling Haaland,Manchester City,Arsenal,1,33,4.212113,33.610771
3766,Erling Haaland,Manchester City,Fulham,0,34,5.416113,39.026886
3767,Erling Haaland,Manchester City,Leeds United,1,35,7.036281,46.063168


Points per game: 5.119999885559082


In [17]:
predictions.to_csv(f'../point_projections/xP_projections_week{gameweek}.csv')

# Collective predictions

Here we use all the 10 models to make collective predictions (mean prediction)

In [18]:
%%time
# run xP projections for each player by using each model and taking the average
my_predictions = []
my_std = []
for ix in prediction_data.index:
    model_predictions = []
    for model_ix in range(10):
        model_predictions.append(xgb_models[model_ix].predict(prediction_data.loc[ix:ix, keep])[0])
    my_predictions.append(np.mean(model_predictions))
    my_std.append(np.std(model_predictions))

CPU times: user 19min 22s, sys: 4min 53s, total: 24min 16s
Wall time: 3min 25s


In [19]:
collective_predictions = predictions.copy()
collective_predictions['xP'] = my_predictions
collective_predictions['std'] = my_std
collective_predictions.head(5)

,name,team,opp_team,was_home,gameweek,xP,std
0,Granit Xhaka,Arsenal,Bournemouth,1,26,3.605445,0.220236
1,Granit Xhaka,Arsenal,Fulham,0,27,3.195131,0.224736
2,Granit Xhaka,Arsenal,Crystal Palace,1,28,3.579107,0.198305
3,Granit Xhaka,Arsenal,Leeds United,1,29,3.320841,0.155277
4,Granit Xhaka,Arsenal,Liverpool,0,30,2.510845,0.133115


In [20]:
aux = collective_predictions[collective_predictions['name']=='Erling Haaland']
display(aux)
ppg = aux['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')

,name,team,opp_team,was_home,gameweek,xP,std
3759,Erling Haaland,Manchester City,Newcastle Utd,1,26,4.865784,0.720001
3760,Erling Haaland,Manchester City,Crystal Palace,0,27,5.130757,0.611321
3761,Erling Haaland,Manchester City,Liverpool,1,29,4.320444,0.724365
3762,Erling Haaland,Manchester City,Southampton,0,30,5.415514,0.602717
3763,Erling Haaland,Manchester City,Leicester City,1,31,6.253116,0.542090
3764,Erling Haaland,Manchester City,Brighton,0,32,4.039344,0.612698
3765,Erling Haaland,Manchester City,Arsenal,1,33,4.201942,0.751674
3766,Erling Haaland,Manchester City,Fulham,0,34,5.361454,0.607373
3767,Erling Haaland,Manchester City,Leeds United,1,35,5.762286,0.721775


Points per game: 5.039999961853027


In [21]:
collective_predictions.to_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv')

# Additional/alternative projections

Eearly season, the model has very little data on some new player, e.g. Haaland, so I'll supplement the data by giving him Kane's statistics for history and then updating those with new Haaland data as it comes

In [ ]:
def alternative_projections(original_player_name, mock_player_name, original_team):
    
    player_name = mock_player_name
    real_player_name = original_player_name
    team = original_team
    aux1 = player_data.loc[player_data['name']==player_name, keep].tail(1)
    aux2 = player_data.loc[player_data['name']==real_player_name, columns_for_ewa+keep]
    
    ewa_intervals = [5,10,20,40]
    for interval in ewa_intervals:
        for col in columns_for_ewa:
            aux2[f'{col}_ewa_{interval}_shifted_0'] = (1-1/interval)*aux1[f'{col}_ewa_{interval}_shifted_0'].values[0] +\
                                                        (1/interval)*aux2[col].values[0]
            
    # mock predictions 
    first_projected_gameweek=gameweek+1
    horizon=10

    aux=aux2.copy()
    all_predictions = []
    aux_prediction_data = pd.DataFrame()
    for count, week in enumerate(np.arange(first_projected_gameweek, first_projected_gameweek+horizon)):
        home_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['home_team']==team)]
        away_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['away_team']==team)]
        for ix in home_games.index:
            aux['was_home'] = 1
            aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
            opp_team = fpl_fixtures.loc[ix, 'away_team']
            aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
            aux_prediction_data = pd.concat([aux_prediction_data,aux])
            xP = xgb_models[count].predict(aux[keep])[0]
            all_predictions.append([real_player_name, team, opp_team, 1, week, xP])
        for ix in away_games.index:
            aux['was_home'] = 0
            aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
            opp_team = fpl_fixtures.loc[ix, 'home_team']
            aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
            aux_prediction_data = pd.concat([aux_prediction_data,aux])
            xP = xgb_models[count].predict(aux[keep])[0]
            all_predictions.append([real_player_name, team, opp_team, 0, week, xP])

    all_predictions = pd.DataFrame(all_predictions, columns=my_columns)
    
    return all_predictions

In [ ]:
def add_alternative_projections(real_player_name, predictions, aux_prediction_data):
    
    predictions = predictions.drop(predictions[predictions['name']==real_player_name].index)
    predictions = pd.concat([predictions, all_predictions])
    predictions = predictions.reset_index(drop=True)
    
    return predictions 

In [ ]:
columns_for_ewa = ['total_points', 'xPoints', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
                   'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
                   'penalties_saved', 'saves', 'threat', 'yellow_cards', 
                'Performance_Sh', 'Performance_SoT', 'Performance_Touches', 'Performance_Press', 'Performance_Tkl',
                  'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG', 'Expected_xA',
                  'SCA_SCA', 'Passes_Cmp', 'Passes_Cmp%', 'Passes_Prog', 'Carries_Carries', 'Carries_Prog',
                  'Dribbles_Succ', 'Dribbles_Att', 'General_KP', 'Touches_Att 3rd', 'Touches_Att Pen', 
                   'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Won%']

In [ ]:
original_player_name = 'Erling Haaland'
mock_player_name = 'Harry Kane'
original_team = 'Manchester City'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
ppg = aux_prediction_data['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')
display(aux_prediction_data)

In [ ]:
original_player_name = 'Ivan Perisic'
mock_player_name = 'Andrew Robertson'
original_team = 'Tottenham'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
aux_prediction_data

In [ ]:
original_player_name = 'Darwin Nunez'
mock_player_name = 'Harry Kane'
original_team = 'Liverpool'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
aux_prediction_data

In [ ]:
#predictions.to_csv(f'../data/xP_projections_week{gameweek}.csv')

# Shap-values for individual projections

In [ ]:
player_name = 'Erling Haaland'
ixs = predictions.loc[predictions['name']==player_name].index
predictions.loc[predictions['name']==player_name]

In [ ]:
explainer = shap.Explainer(xgb_models[0], prediction_data[keep])
shap_values = explainer(prediction_data[keep])

In [ ]:
shap.plots.waterfall(shap_values[ixs[0]], max_display=20)

In [ ]:
prediction_data[keep].loc[ixs[0]:ixs[0]]['bps_ewa_5_shifted_0']